# Sistema de recomendación de anime

In [9]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

#Barra de progreso
import time
from tqdm import tqdm

El objetivo es aplicar todo lo que hemos aprendido hasta el momento, especialmente sobre gradiente descendiente. Para esto desarrollaremos un sistema de recomendación usando factorización de matrices.

En esta entrega se darán todos elementos necesarios para desarrollar un sistema de recomendación de amines, el objetivo es que usted entienda como funciona y desarrolle una solución.

Si desea conocer mas sobre el dataset que usaremos puede revisar acá: https://www.kaggle.com/CooperUnion/anime-recommendations-database

1- Cargue las bases de datos de calificaciones, usuarios y animes. Puede usar para este objetivo la biblioteca pandas. Genere el conjunto de entrenamiento usando los usuarios cómo filas y los animes cómo columnas. Tenga en cuenta que para el algoritmo es necesario que los -1 sean cero, sin embargo, para la recomendación si debe tener en cuenta dichos valores.

In [10]:
#Obtener Listado .CSV
def obtenerListas():
    df = pd.read_csv('anime.csv', delimiter=',', parse_dates=[6], header=0,index_col=False, squeeze=False)
    uf = pd.read_csv('rating.csv', delimiter=',', parse_dates=[2], header=0,index_col=False, squeeze=False)
    return np.array(df),np.array(uf)

d,u=obtenerListas()

#Obtener la cantidad total de animes y usuarios
cantAnimes=len(d)
cantUsers=int(u[-1,0])

animes=np.zeros(len(d))
aux=0
for i in tqdm(range(len(u))): 
    if u[i,1] <= cantAnimes:
        if int(u[i,2])==10 and not(int(u[i,1]) in animes):
            animes[aux] = int(u[i,1])
            aux=aux+1
animes=animes[:aux]
animes.sort()
animes=animes.astype(int)


R = np.zeros((cantUsers,aux),dtype=np.int8)
for i in tqdm(range(len(u))): 
    if u[i,1] <= cantAnimes:
        if int(u[i,1]) in animes and int(u[i,2])>0:
            R[u[i,0]-1,np.where(animes==int(u[i,1]))[0]] = int(u[i,2])
        
print(R)
print(R.shape)

100%|█████████████████████████████████████████████████████████████████████| 7813737/7813737 [01:28<00:00, 87864.61it/s]

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [10 10 10 ...  0  0  8]
 [ 0  0  0 ...  0  0  0]]
(73516, 4850)


2- Agregue la función donde se ejecuta el algoritmo de factorización de matrices. Puede basarse en el algoritmo compartido en las dispositivas. No debe de gastar todas las interaciones. 

In [ ]:
def matrix_factorization(R, P, Q, K, steps=100, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in tqdm(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.5:
            break
    return P, Q.T

3- Programe una función para generar $\hat{R}$. Recuerde que para generar $\hat{R}$ debe realizar los siguiente pasos:

    - Generar R usando el algoritmo de vecinos más cercanos.
    - Defina K.
    - Defina P y Q.
    - Generar R prima.


In [11]:
def vecinoMasCercano(calcular,k,matriz):
    if calcular:
        nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto',n_jobs=-1).fit(matriz)
        distances, indices = nbrs.kneighbors(R)
        return distances,indices
    else:
        distances=np.loadtxt('distances.txt')
        indices=np.loadtxt('indices.txt')
        return distances,indices



4- Realice las siguiente recomendaciones (animes que no ha visto con calificación 10):

    1- Usuario de id 51, películas.
    2- Usuario de id 510, animes con menos de 31 episodios.
    3- Usuario de id 17000, animes con rankins mayores o iguales a 9.
    4- Usuario de id 27005, animes de comedia y aventura.
    5- Usuario de id 37502, TV.
    6- Usuario de id 57502, películas y OVAS.
    7- Usuario de id 67501.

Cada recomendación debe tener como respuesta, número de iteraciones, número de recomendaciones y nombre de los animes,

In [ ]:
def recomendar(kP,idKP,nbrs,R,d,recomRating,idUser,genre,tipo,episodios,rating,members):
    user=np.empty((1,len(kP[0])))
    pos=0
    for i in idKP:
        if idUser in i:
            user=kP[pos]
            break
        pos=pos+1
    pos=0    
    recomendacion=np.empty((len(d),len(d[0])))  
    for i in tqdm(range(len(d))):
        if user[i]>=recomRating:
            boolean=True
            if genre!=None and not(genre in d[i,2]):
                boolean=False
            if tipo!=None and not(tipo == d[i,3]):
                boolean=False  
            if episodios!=None and not(episodios == d[i,4]):
                boolean=False
            if rating!=None and not(rating <= d[i,5]):
                boolean=False  
            if members!=None and not(members <= d[i,5]):
                boolean=False 
            if boolean:
                recomendacion[pos]=d[i]
                pos=pos+1
    return recomendacion[:pos,:]

In [49]:
pos=0
a=42248
b=7473
for i in range(len(R[0])-1):
    if R[a,i] != R[b,i]:
            print('User1',R[a,i],'User2',R[b,i])
            pos=pos+1
print(pos)

0


In [45]:
def optimizarMatriz(R,indices,distances):
    userIguales=np.array([]).reshape(0,2)
    cont=0
    pos=-1
    Raux=np.zeros((len(R),len(R[0])),dtype=np.int8)
    for i in tqdm(range(len(indices))):
        for j in range(len(indices[0])):
            if int(distances[i,j]) == 0:
                index=int(indices[i,j])
                aux1=np.where(userIguales[:,1]==i)
                if i != index:
                    aux2=np.where(userIguales[:,1]==index)
                    if len(aux1[0]) == 0:
                        if len(aux2[0]) == 0:
                            pos=pos+1
                            userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,i]), 0)
                            userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,index]), 0)
                            Raux[cont]=R[i]
                            cont=cont+1
                        else:
                            userIguales=np.insert(userIguales,aux2[0][-1]+1, np.array([userIguales[aux2[0][-1],0],i]), 0)
                    else:
                        if len(aux2[0]) == 0:
                            userIguales=np.insert(userIguales,aux1[0][-1]+1, np.array([userIguales[aux1[0][-1],0],index]), 0)                  
                else:
                    if len(aux1[0]) == 0:
                        pos=pos+1
                        userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,i]), 0)
                        Raux[cont]=R[i]
                        cont=cont+1
    return Raux[:cont,:], userIguales

distances,indices=vecinoMasCercano(False,None,None)
Raux, userIguales = optimizarMatriz(R,indices,distances)
print(Raux.shape)
print(Raux)
print(userIguales.shape)
print(userIguales)
np.savetxt('userIguales3.txt',userIguales)

100%|██████████████████████████████████████████████████████████████████████████| 73516/73516 [00:12<00:00, 5731.96it/s]


(66385, 4850)
[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [10 10 10 ...  0  0  8]
 [ 0  0  0 ...  0  0  0]]
(73516, 2)
[[0.0000e+00 0.0000e+00]
 [1.0000e+00 1.0000e+00]
 [1.0000e+00 2.2760e+03]
 ...
 [6.6382e+04 7.3513e+04]
 [6.6383e+04 7.3514e+04]
 [6.6384e+04 7.3515e+04]]


In [10]:
import csv

with open('userIguales.csv', 'w', newline='\n', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile,delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerows(userIguales)